In [1]:
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
import numpy as np
import pandas as pd
import time
import jieba
import re
import types
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers.core import Lambda
from keras.layers.merge import concatenate, add, multiply
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
import warnings
import io
import sys
import urllib.request
from keras.preprocessing.text import Tokenizer

/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [3]:
# 创建停用词list  
def stopwordslist(stop_file):  
    stopwords = [line.strip() for line in open(stop_file, 'r').readlines()]  
    return stopwords

In [4]:
def cal_word_edit(lhs, rhs):
    len_1 = len(lhs)
    len_2 = len(rhs)
    dist_table = [[0] * (len_2 + 1) for i in range(len_1 + 1)]
    for i in range(len_1 + 1):
        dist_table[i][0] = i
    for j in range(len_2 + 1):
        dist_table[0][j] = j
    for i in range(1, len_1 + 1):
        for j in range(1, len_2 + 1):
            if lhs[i - 1] == rhs[j - 1]:
                cost = 0
            else:
                cost = 1
            deletion = dist_table[i - 1][j] + 1
            insertion = dist_table[i][j - 1] + 1
            substitution = dist_table[i - 1][j - 1] + cost
            dist_table[i][j] = min(min(deletion, insertion), substitution)
    return dist_table[len_1][len_2]

In [5]:
# 对句子进行分词;赋予标签
def seg_sentence(input_file, log_file, stop_file, segwithoutlabel_file):
    words = []
    labels = []
    text1 = []
    outstr=''
    stopwords = stopwordslist(stop_file)  # 这里加载停用词的路径  
    outf = open(log_file, 'w', encoding='utf-8', errors='ignore')
    outft = open(segwithoutlabel_file, 'w', encoding='utf-8', errors='ignore')
    with open(input_file,'r', encoding='utf-8', errors='ignore') as inf:
        for line in inf.readlines():
            line = line.strip()
            # print ("line:",line)
            if line:
                t_vec = line.split('\t')
                if len(t_vec) >= 3:
                    dec = t_vec[0]
                    dec = dec.strip()
                    count_name = t_vec[1]
                    label = str(t_vec[2])
                    if label == 'ILf':
                        label = 'ILF'
                    if label == 'EIf':
                        label = 'EIF'
                else:
                    continue
            labels.append(label)
            # print ("dec", dec)
            # print ("count_name", count_name)
            # print ("label", label)
            sentence_seged = jieba.cut(dec)
            count_seged = jieba.cut(count_name)
            sentence_result = []
            count_result = []
            for word in sentence_seged:
                sentence_result.append(word)
            for count_word in count_seged:
                count_result.append(count_word)
            text2 = []
            # print ("sentence_result", sentence_result)
            # print ("count_result", count_result)
            for word in sentence_result:
                flag = 'O'
                text1 = []
                if word not in stopwords:
                    #print ("stopwords")
                    if word != '\t' and word != '' and word != ' ':
                        #print ("kongbai")
                        for count_e in count_result:
                            #print ("daozhelile")
                            if count_e == '':
                                #print ("count_word", count_e)
                                continue
                            else:
                                if word == count_e:
                                    flag = label
                                    break
                                else:
                                    # print ("word", word)
                                    # print ("count_word", count_e)
                                    # print ("length", len(count_e))
                                    dist = cal_word_edit(word, count_e)
                                    min_len = min(len(word), len(count_e))
                                    if min_len <= 3:
                                        if dist < 2:
                                            flag = label
                                            break
                                    elif min_len <= 7:
                                        if dist < 3:
                                            flag = label
                                            break
                                    else:
                                        if dist < 4:
                                            flag = label
                                            break
                        i = 0
                        for str_tmp in word:
                            if flag == 'O':
                                str_flag = 'O'
                            elif i == 0:
                                str_flag = 'B' + '-' + flag
                            else:
                                str_flag = 'I' + '-' + flag
                            i += 1
                            text1.append(str_tmp)
                            text1.append(str_flag)
                            text2.append(str_tmp)
                            outstr = str_tmp.strip()
                            outstr = outstr.strip(' ')
                            outf.write(outstr)
                            outft.write(outstr)
                            outf.write(" ")
                            outft.write(" ")
                            outf.write(str_flag)
                            outf.write("\n")
            outf.write("]")
            outf.write("\n")
                        #print ("line:",outstr)
    outf.close()
    outft.close()
    return text2

In [6]:
locale = True
addh = ""
if locale:
    addh = "/mnt/d"

In [8]:
train_text=seg_sentence(
    addh+'/My Drive/Graduation/old/实验/summary_2.txt',
    addh+'/My Drive/Graduation/old/实验/test_data_e.data',
    addh+'/My Drive/Graduation/old/实验/stopwords.txt',
    addh+'/My Drive/Graduation/old/实验/segwithoutlabel_file.txt'
)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.173 seconds.
Prefix dict has been built successfully.
